In [1]:
import os
import subprocess
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from langchain_community.llms import Ollama # will be used for prompting
from langchain.vectorstores import Chroma # will be used for vectordb store
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document # will be used to store text in vector store 


In [2]:
llm = Ollama(model="llama3")
embedding = OllamaEmbeddings(model="llama3")
PERSIST_DIR = "vectordb"
db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_25498/1332557208.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")
/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_25498/1332557208.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)


In [3]:
def get_most_recent_unprocessed_idea(db):
    results = db.get(include=["documents", "metadatas"])

    unprocessed = [
        (meta["timestamp"], Document(page_content=doc, metadata=meta))
        for meta, doc in zip(results["metadatas"], results["documents"])
        if not meta.get("processed", False)
    ]

    if not unprocessed:
        print("No unprocessed ideas found.")
        return None

    unprocessed_sorted = sorted(unprocessed, key=lambda x: x[0], reverse=True)
    _, document = unprocessed_sorted[0]

    print(f"Most recent unprocessed idea ID: {document.metadata.get('id')}")
    return document


In [4]:

most_recent_idea = get_most_recent_unprocessed_idea(db)
if most_recent_idea:
    print(most_recent_idea.page_content) 


Most recent unprocessed idea ID: 1b6c1749-2bfe-4078-bd43-1a660dec2099
"Hey, have you ever struggled with building a mobile app that can handle real-time analytics and data visualizations? Yeah, I know what it's like - it's like trying to solve a puzzle blindfolded! But fear not, my friends, because Amazon QuickSight is here to save the day. It's a fast, easy-to-use business intelligence service that lets you create stunning dashboards and reports in minutes. So why would you use something like this? Well, imagine you're building an app for sports teams to track their performance metrics - QuickSight makes it easy to visualize complex data and spot trends.

To set it up, you can just upload your data and let the algorithms do the heavy lifting. No coding required! And did you know that many companies are already using QuickSight to gain insights from their data? Yeah, it's that powerful. So if you're building something cool with data and want to unlock its secrets, give Amazon QuickSigh

In [5]:
def generate_video_title(script):
    llm = Ollama(model="llama3")
    response = llm.invoke(f"You are a helpful assistant. You really know your stuff. Generate a title for the following script: {script} Keep it short and concise something liket this  AWS Tip: CloudFront Made Simple. I need to make sure it is something that will be interesting to the audience and only 6 words.Give me the title only, no other text. Nothing else. literally. choose the best title for the script. no quotes even just the text. please make sure no quotes or anything else. just the text.")
                        
    return response

def generate_video_description(script):
    llm = Ollama(model="llama3")
    response = llm.invoke(f"Generate a description for the following script: {script} Keep it short and concise something like this: Here's how CloudFront boosts your AWS app performance in seconds. Learn, deploy, repeat. #AWS #Tech. Do not include any other text. Nothing else. literally. choose the best description for the script. Choose the best description for the script. I only want one. Give me one response. no quotes even just the text. please make sure no quotes or anything else. just the text.")
    return response

In [6]:
title = generate_video_title(most_recent_idea.page_content)
description = generate_video_description(most_recent_idea.page_content)

print(title)
print(description)

Unleash Data Insights with Amazon QuickSight
Solving data puzzles without coding required: Unlock Amazon QuickSight's insights with ease.


In [7]:
def trim_video(input_path, output_path="output_to_be_up.mp4", max_seconds=60):
    subprocess.run([
        "ffmpeg",
        "-i", input_path,
        "-t", str(max_seconds),
        "-vf", "scale=1080:1920",  # Ensure vertical 9:16
        "-c:a", "aac",
        "-c:v", "libx264",
        "-preset", "ultrafast",
        output_path
    ], check=True)
    return output_path

In [8]:
def upload_to_youtube_short(video_path, title, description):
    scopes = ["https://www.googleapis.com/auth/youtube.upload"]
    
    flow = InstalledAppFlow.from_client_secrets_file("client_secrets.json", scopes)
    creds = flow.run_local_server(port=0)


    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=creds)

    request_body = {
        "snippet": {
            "categoryId": "22",  
            "title": f"{title} #Shorts",
            "description": description
        },
        "status": {
            "privacyStatus": "public"
        }
    }

    media_file = MediaFileUpload(video_path)

    request = youtube.videos().insert(
        part="snippet,status",
        body=request_body,
        media_body=media_file
    )
    
    response = request.execute()
    print(f"Uploaded to: https://youtube.com/watch?v={response['id']}")

In [9]:
# === USAGE ===
trimmed_path = trim_video("output_final.mp4")  # or use your full path here
upload_to_youtube_short(
    video_path=trimmed_path,
    title=title,
    description=description
)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=142192968968-ocfps0dpa665h9glnfis4mlhmlt34abi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61703%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=Ft304CnvUBcgzTtOuftIIG5gZsJ1TV&access_type=offline
✅ Uploaded to: https://youtube.com/watch?v=d-E4wpEHywo


In [10]:
def delete_output_and_subway_files(directory="."):
    for filename in os.listdir(directory):
        if filename.startswith("output") or filename.startswith("subway"):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# Run it
delete_output_and_subway_files()


🗑️ Deleted: ./output_to_be_up.mp4
🗑️ Deleted: ./output_final.mp4
🗑️ Deleted: ./subway.mp4
🗑️ Deleted: ./output.srt
🗑️ Deleted: ./output.wav
